In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import json
import math
import copy
import random
from collections import defaultdict, Counter, OrderedDict
from itertools import combinations
import unicodedata
import re
import matplotlib.pyplot as plt
import pickle

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
import Levenshtein as lvst
from metaphone import doublemetaphone as dmt_phone
import networkx as nx
import nxviz as nv

In [5]:
from sparse_dot_topn import awesome_cossim_topn as cossim_topn

In [6]:
import spacy
from spacy import displacy
from spacy.symbols import nsubj, VERB, NOUN
from spacy.tokens import Doc, Token, Span
from spacy.attrs import ENT_IOB, ENT_TYPE
from spacy.tokenizer import Tokenizer

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
from Normalization_Helper import get_df_match,add_match_score,cut_off,get_connected_comp,get_postProcessigUnits
from Normalization_Helper import choose_normalized,get_comp_final,make_norm_dict,normalize_unicode_to_ascii

In [9]:
import math

In [10]:
df = pd.read_csv("Final_Spacy_10000_id2.csv")

In [11]:
# df = pd.read_excel('Cleaned_Spacy_10000_id2.xlsx')

In [43]:
df.head()

,Person_Spacy,Person_Spacy_id,Organization_Spacy,Organization_Spacy_id,Location_Spacy,Location_Spacy_id
0,Shaquille O'Neal,['167424f8-ca7f-4224-a00b-a54a5a7766c7'],NBA,"['167424f8-ca7f-4224-a00b-a54a5a7766c7', 'a6f6...",Los Angeles,"['167424f8-ca7f-4224-a00b-a54a5a7766c7', '4b80..."
1,Wilt Chamberlain's,['167424f8-ca7f-4224-a00b-a54a5a7766c7'],the Houston Rockets,['167424f8-ca7f-4224-a00b-a54a5a7766c7'],Rockets,['167424f8-ca7f-4224-a00b-a54a5a7766c7']
2,Muhammad Ali,"['167424f8-ca7f-4224-a00b-a54a5a7766c7', 'f783...",Olajuwon,['167424f8-ca7f-4224-a00b-a54a5a7766c7'],Jordan,"['167424f8-ca7f-4224-a00b-a54a5a7766c7', '6da7..."
3,Chamberlain,"['167424f8-ca7f-4224-a00b-a54a5a7766c7', '9841...",Russel,['167424f8-ca7f-4224-a00b-a54a5a7766c7'],Chicago,"['167424f8-ca7f-4224-a00b-a54a5a7766c7', '7185..."
4,LeBon James,['167424f8-ca7f-4224-a00b-a54a5a7766c7'],the Boston Celtics,"['167424f8-ca7f-4224-a00b-a54a5a7766c7', '8a8c...",Lew Alcindor,['167424f8-ca7f-4224-a00b-a54a5a7766c7']


In [44]:
type(df)

pandas.core.frame.DataFrame

In [12]:
persons = df['Person_Spacy']
organization = df['Organization_Spacy']
location = df['Location_Spacy']

In [13]:
print(len(persons),len(organization),len(location))

53158 53158 53158


In [14]:
persons.dropna(inplace=True,axis=0)
organization.dropna(inplace=True,axis=0)
location.dropna(inplace=True,axis=0)

In [15]:
print(len(persons),len(organization),len(location))

53158 40729 14010


In [16]:
persons = list(persons)
organization = list(organization)
location = list(location)

In [49]:
def clean_text_p(data, lowercasing=False, special_char=True, convert2ascii=True):
   
    if convert2ascii:
        normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
        val = normal.decode("utf-8")
    else:
        val = data
    
    if lowercasing:
        val = val.lower()
    
    if special_char: # remove special characters
        val = re.sub('[^A-Za-z. ]+', ' ', val) #0-9
    
    # remove multiple spaces
    val = re.sub(' +', ' ', val)
    
    return val

def clean_text(data, lowercasing=False, special_char=True, convert2ascii=True):
    
    
    if convert2ascii:
        normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
        val = normal.decode("utf-8")
    else:
        val = data
    
    if lowercasing:
        val = val.lower()
    
    if special_char: # remove special characters
        val = re.sub('[^A-Za-z0-9.\' ]+', ' ', val) #0-9
    
    # remove multiple spaces
    val = re.sub(' +', ' ', val)
    
    return val

In [50]:
persons = [clean_text_p(i) for i in persons]
organization = [clean_text(i) for i in organization]
location = [clean_text(i) for i in location]

### PERSONS

In [51]:
df_persons = get_df_match(persons,entity_type=None, sorting_1st_letters=True)

# of candidate pairs: 13642


In [52]:
weights_person = [4,2,1,1,1]

In [53]:
add_match_score(df_persons, weights=weights_person)

In [54]:
df_persons = cut_off(df_persons)

In [55]:
df_persons.shape

(3826, 8)

In [56]:
df_persons.tail(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
3816,de que los,que los,0.830699,False,1.233946e-04,0.333333,0.804762,0.495668
3817,los policias,Policias,0.840839,False,8.315280e-07,0.250000,0.847222,0.495620
3818,Hernandez,Hernan Fernandez,0.855434,False,1.233946e-04,0.142857,0.895370,0.495565
3819,Dan Stanley,Stanley,0.856587,False,1.233946e-04,0.250000,0.783550,0.495558
3820,Programm,Programm zum,0.808011,False,1.233946e-04,0.250000,0.977778,0.495549
3821,Vela,M. Vela,0.767725,False,1.233946e-04,0.500000,0.888889,0.495546
3822,Theresa,Theresa May,0.811549,False,8.315280e-07,0.250000,0.963636,0.495537
3823,X. Du,H.X. Du,0.788468,False,1.233946e-04,0.500000,0.805556,0.495506
3824,Muhammad bin Salman bin Abd alAziz Al Saud,Crown Prince Muhammad bin Salman bin Abd alAzi...,0.903010,False,8.315280e-07,0.076923,0.770418,0.495487
3825,Carl Williams,Pearl Williams,0.761269,False,1.233946e-04,0.500000,0.914141,0.495482


In [57]:
comp_persons = get_connected_comp(df_persons)

In [58]:
comp_persons_short, comp_persons_long, comp_persons_all = get_postProcessigUnits(comp_persons)

In [59]:
print(len(comp_persons_short))
print(len(comp_persons_long))

2420
534


In [60]:
comp_persons_all

[(0, [' . ', ' ...', ' . . ', '. ', '.', ' .', ' ']),
 (1, [' . Brandon Williams', 'Brandon Williams']),
 (2, [' . bn', ' bn']),
 (3, [' Apr ', 'Apr ']),
 (4, [' Bid Deadline', 'Bid Deadline']),
 (5, [' Boris', 'Boris']),
 (6, [' Clarissa Gonzalez', 'Clarissa Gonzalez']),
 (7, [' E. Johnson Ave.', 'E. Johnson Ave.']),
 (8, [' Financier', 'Financier']),
 (9, [' Floyd', 'Floyd']),
 (10, [' Goods', 'Goods']),
 (11, [' Jul ', 'Jul ']),
 (12, [' Karla', 'Karla']),
 (13, [' MT USA', 'MT USA']),
 (14, [' Moore', 'Moore']),
 (15, [' N. Miami Ave.', 'N. Miami Ave.']),
 (16, [' Niles North', 'Niles North']),
 (17, [' Ocoee', 'Ocoee']),
 (18, [' RISKs', 'Risks']),
 (19, [' YouTube', 'YouTube', 'Youtube']),
 (20, [' lncRNA H ', ' LncRNA H ']),
 (21, ['A s', 'A. S.']),
 (22, ['A. Bruno', 'Bruno']),
 (23, ['A. Ciccozzi', 'M. Ciccozzi']),
 (24, ['A. Hamosh ', 'A. Hamosh']),
 (25, ['A. J.', 'A.J.']),
 (26, ['A. L.', 'A L ']),
 (27, ['A. Muraro ', 'A. Muraro']),
 (28, ['A. Peyraud', 'Armelle Peyraud'])

In [65]:
comp_persons_final = get_comp_final(comp_persons_all, comp_persons_short, is_all=True)

In [66]:
normalized_persons = choose_normalized(comp_persons_final,choice='long') #, choice='long'

In [67]:
persons_raw2norm, persons_norm2raw = make_norm_dict(comp_persons_final, normalized_persons)

In [68]:
persons_norm2raw

defaultdict(list,
            {' . . ': [' . ', ' ...', ' . . ', '. ', '.', ' .', ' '],
             'Brandon Williams': [' . Brandon Williams', 'Brandon Williams'],
             ' . bn': [' . bn', ' bn'],
             'Apr ': [' Apr ', 'Apr '],
             'Bid Deadline': [' Bid Deadline', 'Bid Deadline'],
             'Boris': [' Boris', 'Boris'],
             'Clarissa Gonzalez': [' Clarissa Gonzalez', 'Clarissa Gonzalez'],
             'E. Johnson Ave.': [' E. Johnson Ave.', 'E. Johnson Ave.'],
             'Financier': [' Financier', 'Financier'],
             'Floyd': [' Floyd', 'Floyd'],
             'Goods': [' Goods', 'Goods'],
             'Jul ': [' Jul ', 'Jul '],
             'Karla': [' Karla', 'Karla'],
             'MT USA': [' MT USA', 'MT USA'],
             'Moore': [' Moore', 'Moore'],
             'N. Miami Ave.': [' N. Miami Ave.', 'N. Miami Ave.'],
             'Niles North': [' Niles North', 'Niles North'],
             'Ocoee': [' Ocoee', 'Ocoee'],
           

### Organizations

In [69]:
df_organizations = get_df_match(organization,entity_type=None, sorting_1st_letters=True, partial_1st_letter_match=True)

# of candidate pairs: 10063


In [70]:
weights_organizations = [2, 4, 0.5, 0.5, 0.5]

In [71]:
add_match_score(df_organizations, weights=weights_organizations)

In [72]:
comp_organizations = get_connected_comp(df_organizations)

In [73]:
comp_organizations_short, comp_organizations_long, comp_organizations_all = get_postProcessigUnits(comp_organizations)

In [74]:
comp_organizations_all

[(0, [' . ', '.', '. ', ' .', 'Director ', ' ']),
 (1, [' AT Kearney India P Ltd', 'Kearney India P Ltd']),
 (2, [' Airfinance Journal s', ' Airfinance Journal', ' Airfinance Journal ']),
 (3, [' Apoorva Systems', 'Apoorva Systems']),
 (4, [' Court', 'General Court of']),
 (5,
  [' INN Resource https twitter.cominn resource',
   ' INN Resource https twitter.comINN Resource']),
 (6, [' Market Overview', 'Overview']),
 (7, [' Tata Elxsi Ltd', 'Tata Elxsi Ltd']),
 (8, [' WJHL11', ' WJHL11 ABCTriCities']),
 (9, ['1 5 Prozent des BIP', 'Prozent des BIPs zulegen', 'Prozent des BIP']),
 (10, ['11th Force Support Squadron', 'the 11th Force Support Squadron s']),
 (11, ['19 30 00 Appeals', '13 30 00 Appeals', '12 30 00 Appeals']),
 (12, ['2nd Congressional District', 'Congressional District 1']),
 (13, ['2nd Space Warning Squadron', 'Space Warning Squadron']),
 (14, ['34 cup', '34 cup barbecue']),
 (15, ['344th St. Traffic', '344th St.']),
 (16, ['3540 Main Hwy.', '3540 Main Hwy']),
 (17, ['3rd

In [75]:
comp_organizations_final = get_comp_final(comp_organizations_all, comp_organizations_short, is_all=True)

In [76]:
len(comp_organizations_final)

5421

In [77]:
normalized_organizations = choose_normalized(comp_organizations_final,choice='long') #, choice='long'

In [78]:
organizations_raw2norm, organizations_norm2raw = make_norm_dict(comp_organizations_final, normalized_organizations)

In [79]:
organizations_norm2raw

defaultdict(list,
            {'Director ': [' . ', '.', '. ', ' .', 'Director ', ' '],
             'Kearney India P Ltd': [' AT Kearney India P Ltd',
              'Kearney India P Ltd'],
             ' Airfinance Journal s': [' Airfinance Journal s',
              ' Airfinance Journal',
              ' Airfinance Journal '],
             'Apoorva Systems': [' Apoorva Systems', 'Apoorva Systems'],
             'General Court of': [' Court', 'General Court of'],
             ' INN Resource https twitter.comINN Resource': [' INN Resource https twitter.cominn resource',
              ' INN Resource https twitter.comINN Resource'],
             'Overview': [' Market Overview', 'Overview'],
             'Tata Elxsi Ltd': [' Tata Elxsi Ltd', 'Tata Elxsi Ltd'],
             ' WJHL11 ABCTriCities': [' WJHL11', ' WJHL11 ABCTriCities'],
             'Prozent des BIPs zulegen': ['1 5 Prozent des BIP',
              'Prozent des BIPs zulegen',
              'Prozent des BIP'],
             'the 

In [80]:
[organizations_norm2raw.items()]

[dict_items([('Director ', [' . ', '.', '. ', ' .', 'Director ', ' ']), ('Kearney India P Ltd', [' AT Kearney India P Ltd', 'Kearney India P Ltd']), (' Airfinance Journal s', [' Airfinance Journal s', ' Airfinance Journal', ' Airfinance Journal ']), ('Apoorva Systems', [' Apoorva Systems', 'Apoorva Systems']), ('General Court of', [' Court', 'General Court of']), (' INN Resource https twitter.comINN Resource', [' INN Resource https twitter.cominn resource', ' INN Resource https twitter.comINN Resource']), ('Overview', [' Market Overview', 'Overview']), ('Tata Elxsi Ltd', [' Tata Elxsi Ltd', 'Tata Elxsi Ltd']), (' WJHL11 ABCTriCities', [' WJHL11', ' WJHL11 ABCTriCities']), ('Prozent des BIPs zulegen', ['1 5 Prozent des BIP', 'Prozent des BIPs zulegen', 'Prozent des BIP']), ('the 11th Force Support Squadron s', ['11th Force Support Squadron', 'the 11th Force Support Squadron s']), ('12 30 00 Appeals', ['19 30 00 Appeals', '13 30 00 Appeals', '12 30 00 Appeals']), ('Congressional District

In [48]:
df_organizations.tail(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
10053,the US FDA,FDA,0.738316,False,8.315280e-07,0.142857,0.0,0.206408
10054,Schiff ABPAS,ABPAS,0.733251,False,1.233946e-04,0.142857,0.0,0.205066
10055,MVP,NBA MVP,0.705708,False,1.233946e-04,0.250000,0.0,0.204864
10056,The Fig Rig,FIG,0.734546,False,8.315280e-07,0.125000,0.0,0.204212
10057,Gay NJ,NJ,0.702937,False,8.315280e-07,0.250000,0.0,0.204117
10058,KIA SUV,SUV,0.702603,False,1.233946e-04,0.250000,0.0,0.204036
10059,the FFA Club,FFA,0.735565,False,8.315280e-07,0.111111,0.0,0.203558
10060,MAX,Office Max,0.703811,False,1.233946e-04,0.142857,0.0,0.197215
10061,SWAT,the SWAT Team,0.711773,False,8.315280e-07,0.111111,0.0,0.197214
10062,NCAA,the NCAA Council,0.709817,False,8.315280e-07,0.083333,0.0,0.194840


### Locations

In [81]:
df_locations = get_df_match(location, entity_type=None, partial_1st_letter_match=True)

# of candidate pairs: 3255


In [82]:
weights_locations = [4, 2, 0.5, 0.5, 0.5]

In [83]:
add_match_score(df_locations, weights=weights_locations)

In [84]:
df_locations = cut_off(df_locations, q=0.5)

In [85]:
comp_locations = get_connected_comp(df_locations)

In [86]:
comp_locations_short, comp_locations_long, comp_locations_all = get_postProcessigUnits(comp_locations)

In [87]:
comp_locations_all

[(0, [' . ', '.', 'Miss.', ' .', ' ']),
 (1, [' BRCA1 ', 'BRCA1']),
 (2, [' Fleischer', 'Fleischer']),
 (3, [' Rhode Island', 'Rhode Island']),
 (4, [' Singh', 'Singh']),
 (5, ['A A', 'A']),
 (6, ['A.K.', 'A.K']),
 (7, ['A.M.', 'A. M.']),
 (8, ['ABasin s', 'ABasin']),
 (9, ['ADELANTO', 'Adelanto']),
 (10, ['AL', 'al.']),
 (11, ['ALBANY', 'Albany']),
 (12, ['ALTON', 'Alton']),
 (13, ['ANCHORAGE', 'Anchorage']),
 (14, ['ANGOLA', 'Angola', 'Angolan']),
 (15, ['ANKARA', 'Ankara']),
 (16, ['ASHEVILLE', 'Asheville']),
 (17, ['AUSTIN', 'Austin']),
 (18, ['Adams County', 'Ada County']),
 (19, ['Ala', 'Ala.']),
 (20, ['Alamance County', 'ALAMANCE COUNTY', 'Alamance County s']),
 (21, ['Alamance County Jail', 'the Alamance County Jail']),
 (22, ['Alameda County', 'Alameda']),
 (23, ['Albeit', ' albeit']),
 (24, ['Albemarle', 'Albemarle County']),
 (25, ['Algeria', 'Alger']),
 (26,
  ['Allegheny', 'Allegheny River', 'Allegheny County s', 'Allegheny County']),
 (27, ['Alpine', 'Alpina']),
 (28, ['

In [88]:
comp_locations_final = get_comp_final(comp_locations_all, comp_locations_short, is_all=True
                                     )

In [89]:
normalized_locations = choose_normalized(comp_locations_final, choice='short')

In [90]:
locations_raw2norm, locations_norm2raw = make_norm_dict(comp_locations_final, normalized_locations)

In [91]:
locations_norm2raw

defaultdict(list,
            {' ': [' . ', '.', 'Miss.', ' .', ' '],
             'BRCA1': [' BRCA1 ', 'BRCA1'],
             'Fleischer': [' Fleischer', 'Fleischer'],
             'Rhode Island': [' Rhode Island', 'Rhode Island'],
             'Singh': [' Singh', 'Singh'],
             'A': ['A A', 'A'],
             'A.K': ['A.K.', 'A.K'],
             'A.M.': ['A.M.', 'A. M.'],
             'ABasin': ['ABasin s', 'ABasin'],
             'ADELANTO': ['ADELANTO', 'Adelanto'],
             'AL': ['AL', 'al.'],
             'ALBANY': ['ALBANY', 'Albany'],
             'ALTON': ['ALTON', 'Alton'],
             'ANCHORAGE': ['ANCHORAGE', 'Anchorage'],
             'ANGOLA': ['ANGOLA', 'Angola', 'Angolan'],
             'ANKARA': ['ANKARA', 'Ankara'],
             'ASHEVILLE': ['ASHEVILLE', 'Asheville'],
             'AUSTIN': ['AUSTIN', 'Austin'],
             'Ada County': ['Adams County', 'Ada County'],
             'Ala': ['Ala', 'Ala.'],
             'Alamance County': ['Alamance Cou